# <h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import Requirements" data-toc-modified-id="Import-Requirements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Requirements</a></span></li><li><span><a href="#Prepare Training Data" data-toc-modified-id="Prepare-Training-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare Training Data</a></span><ul class="toc-item"></ul></li><li><span><a href="#Model Training" data-toc-modified-id="Model Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model Training</a></span></li><li><span><a href="#Model Saving" data-toc-modified-id="Model Saving-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model Saving</a></span><ul class="toc-item"></ul></li><li><span><a href="#Validation and Results" data-toc-modified-id="Validation and Results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Validation and Results</a></span><ul class="toc-item"></ul></div>

<a id='Import Requirements'></a>

# Import Requirements

In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string
import matplotlib.pyplot as plt

<a id='Prepare Training Data'></a>

# Prepare Training Data

Input data for training consists of both historical data and CICD data( Production run data for which manual agent validation has been done for the ML prediction)

In [2]:
def preprocess_text(message):

    #stopwords
    stpwrd = nltk.corpus.stopwords.words('english')
    #stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]',' ', message.lower())
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    return message

In [3]:
#data_cicd=pd.read_csv('../data/retrain_data/2022-12-13 - INDIRECTTX-1926 - Walgreens_after_tagging.csv' , usecols = ['item_name','subsection_name','description','establishment_type','CAT NAME,CAT_TEMP','Integers'])
#data_cicd['target_new']=data_cicd['CAT NAME,CAT_TEMP'] + ":" + data_cicd['Integers']
#data_cicd.drop(['CAT NAME,CAT_TEMP', 'Integers'],inplace=True,axis=1)
#data_cicd['combined_text'] = final_data[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)


In [13]:
data_cicd=pd.read_csv('../data/retrain_data/TaxML-CICD - Prod_Data (4).csv')

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
data_cicd.shape

(539847, 15)

In [15]:
data_cicd = data_cicd.dropna(axis=0, subset=['Agent Corrected CAT Name'])
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='#REF!'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='106,107,1'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='Bottled Tea Drink'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='Bottled tea drink'].index)
data_cicd = data_cicd.drop(data_cicd[data_cicd['Agent Corrected CAT Name']=='Air Freshener'].index)


In [16]:
data_cicd.shape

(539198, 15)

In [12]:
#xd=data_cicd['Agent Corrected CAT Name'].unique()
#k=pd.Series(xd)
#k.to_csv('k.csv')

In [19]:
mapping_data=pd.read_csv('../data/retrain_data/mapping_latest.csv')
cat_dict = dict(zip(mapping_data['Agent CAT Name'], mapping_data['Updated CAT Name']))
#int_dict = dict(zip(mapping_data['Agent CAT Name'], mapping_data['Updated Integer']))
data_cicd["Agent Corrected CAT Name standarized"] = data_cicd["Agent Corrected CAT Name"].replace(cat_dict)
#data_cicd["Agent Corrected Integers"] = data_cicd["Agent Corrected CAT Name"].replace(int_dict)

In [30]:
#int_dict

In [33]:
#mapping_data_1=pd.read_csv('../data/MASTER_Tax_Category_Mapping_latest1.csv')
#int_dict_1 = dict(zip(mapping_data_1['Agent CAT Name'], mapping_data_1['Updated Integer']))
#data_cicd["Agent Corrected Integer standarized"] = data_cicd["Agent Corrected CAT Name standarized"].map(int_dict)
#data_cicd["Agent Corrected Integer standarized_2"] = data_cicd["Agent Corrected CAT Name"].apply(lambda x:int_dict_1[x])

KeyError: 'CAT_WATER,TRAIT_NONCARB,TRAIT_FLV_SWT,CONTAINER_BOTTLED,TEMP_COLD'

In [28]:
data_cicd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539198 entries, 0 to 539846
Data columns (total 17 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   CICD Run Date                         539198 non-null  object
 1   Date (BOT Sent Details to COE Team)   539198 non-null  object
 2   UniqueUUID                            539198 non-null  object
 3   store_uuid                            539198 non-null  object
 4   item_uuid                             539198 non-null  object
 5   Item                                  539196 non-null  object
 6   Description                           316081 non-null  object
 7   establishment_type                    539198 non-null  object
 8   CAT Name                              539192 non-null  object
 9   Integer                               539192 non-null  object
 10  Confidence Score                      539192 non-null  object
 11  Agent Correct

In [22]:
data_cicd.shape

(539198, 16)

In [23]:
data_cicd.to_csv('data_cicd_cat_mapping.csv')

In [8]:
#read input from cicd data into dataframe
#data_cicd=pd.read_csv('../data/retrain_data/TaxML-CICD - Prod_Data (3).csv', usecols = ['UniqueUUID','Item','Description','establishment_type','Confidence Score','Agent Corrected CAT Name', 'Agent Corrected Integer','CAT NAME_ ValidationScore [0-100]','Integer_ValidationScore[0-100]'])
print(data_cicd.shape)
#misclassified data                                        
data_cicd_misclassification=data_cicd[(data_cicd['CAT NAME_ ValidationScore [0-100]']==0) &(data_cicd['Integer_ValidationScore[0-100]']==0)]
data_cicd_latest=data_cicd_misclassification[['UniqueUUID','Item','Description','establishment_type','Agent Corrected CAT Name', 'Agent Corrected Integer']]
data_cicd_latest['target_new']=data_cicd_latest['Agent Corrected CAT Name'] + ":" + data_cicd_latest['Agent Corrected Integer']
data_cicd_latest.drop(['Agent Corrected CAT Name', 'Agent Corrected Integer'],inplace=True,axis=1)
#mergedStuff = pd.concat([data_df, data_cicd_latest], join="outer")
# combine the columns Item, Description and establishment_type into one column 'combined_text'
#data_cicd_final['combined_text'] = data_cicd_final[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
#data_cicd_final['processed_text']= data_cicd_final['combined_text'].map(lambda s:preprocess_text(s)) 

#data_cicd_final = data_cicd_final.reset_index(drop=True)
# prepare the target column by combining 'Agent Corrected CAT Name' and 'Agent Corrected Integer'

#data_cicd_final=data_cicd_final.drop_duplicates(subset=['processed_text','target_new'],keep='first')
#print(data_cicd_final.shape)

#remove rows having empty target column
#data_cicd_final.dropna(subset=['target_new'],inplace=True)

#data_cicd_final = data_cicd_final[data_cicd_final['target_new']!= '#REF!:#REF!']
#data_cicd_final.to_csv('data_cicd_final.csv')
#X_cicd= data_cicd_final[['Item','Description','establishment_type','processed_text']]
#y_cicd= data_cicd_final['target_new']

# split the cicd data into train and test 
#X_train_cicd, X_test_cicd, y_train_cicd, y_test_cicd = train_test_split(X_cicd, y_cicd,shuffle=True, test_size = .02, random_state = 42)



(613792, 9)


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
#read input from historical data into dataframe
data_df = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data_24_11_22.csv', encoding='utf8',engine='python',usecols=['UniqueUUID','Item','Description','establishment_type','target_new'])

In [10]:
#mergedStuff_cicd= data_df.set_index('UniqueUUID').join(data_cicd_latest.set_index('UniqueUUID'))
final_data=pd.concat([data_df, data_cicd_latest], join="outer")

In [11]:
# combine the columns Item, Description and establishment_type into one column 'combined_text'
final_data['combined_text'] = final_data[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
final_data['processed_text']= final_data['combined_text'].map(lambda s:preprocess_text(s)) 
print(final_data.shape)

(411330, 7)


In [12]:
final_data.head()

,UniqueUUID,Item,Description,establishment_type,target_new,combined_text,processed_text
0,\n,Captain Morgan White Rum.1.75L Bottle,NaN,GROCERY,CAT_LIQUOR:535,Captain Morgan White Rum.1.75L Bottle GRO...,captain morgan white rum bottle grocery
1,\n,D’USSÉ® VSOP Cognac.375ml Bottle,NaN,GROCERY,CAT_LIQUOR:535,D’USSÉ® VSOP Cognac.375ml Bottle GROCERY,us vsop cognac ml bottle grocery
2,\n,Jim Beam Honey Bourbon Whiskey.750ml Bottle,NaN,GROCERY,CAT_LIQUOR:535,Jim Beam Honey Bourbon Whiskey.750ml Bott...,jim beam honey bourbon whiskey ml bottle grocery
3,\n,Ardbeg Scotch Uigeadail. 750ml Bottle,NaN,GROCERY,CAT_LIQUOR:535,Ardbeg Scotch Uigeadail. 750ml Bottle GROCERY,ardbeg scotch uigeadail ml bottle grocery
4,\n,Jameson Irish Whiskey.1.75L Bottle Size,NaN,GROCERY,CAT_LIQUOR:535,Jameson Irish Whiskey.1.75L Bottle Size GR...,jameson irish whiskey bottle size grocery


In [13]:

final_data=final_data.drop_duplicates(subset=['processed_text','target_new'],keep='first')
print(final_data.shape)
#remove rows having empty target column
final_data.dropna(subset=['target_new'],inplace=True)

X= final_data[['Item','Description','establishment_type','processed_text']]
y= final_data['target_new']

# split the cicd data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True, test_size = .20, random_state = 42)


(343033, 7)


We will append the CICD data to the historical data to create the final train and test data.
Train set has 80% of all historical data and 90% of all cicd data.
Test set consists of 20% of historic data and 10% of all cicd data.

In [14]:
#X_train_final = X_train.append(X_train_cicd)
#X_test_final = X_test.append(X_test_cicd)
#y_train_final = y_train.append(y_train_cicd)
#y_test_final = y_test.append(y_test_cicd)

X_train_final = X_train
X_test_final = X_test
y_train_final = y_train
y_test_final = y_test

<a id='Model Training'></a>

In [15]:
print('Training data size: {}'.format(len(X_train_final)))
print('Test data size: {}'.format(len(X_test_final)))

Training data size: 274426
Test data size: 68607


In [16]:
print('Number of unique labels in train data: {}'.format(len(y_train_final.unique().tolist())))
print('Number of unique labels in test data: {}'.format(len(y_test_final.unique().tolist())))

Number of unique labels in train data: 326
Number of unique labels in test data: 285


In [17]:
category_count = data_df.groupby(['target_new'],sort=False).agg({'target_new':'count'})
category_count.rename(columns={'target_new':'count'},inplace=True)
category_count.sort_values('count',ascending=False)

,count
target_new,
CAT_LIQUOR:535,51798
CAT_TPP:531,35117
"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",33507
CAT_WINE:534,24877
CAT_BEER:533,23818
...,...
CAT_PREPACKAGED_FOOD_SALT_PRESERVED_SEA_FOOD:739,4
CAT_POSTAGE:527,2
CAT_PRESCRIPTION_DRUGS:520,2


In [18]:
category_count.to_csv('category_count.csv')

# Model Training

The Model Pipeline consists of 1. CountVectorizer, 2. Tfidf-Transformer 3. MultinomialNB 

In [19]:
vectorizer = CountVectorizer(strip_accents='ascii',token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', stop_words='english', max_df=0.85)
X = vectorizer.fit_transform(X_train_final['processed_text'].values)
features = vectorizer.get_feature_names()
len(features)

83886

In [20]:
# create a result dataframe to store final results
result=X_test_final

#create the model pipeline
rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii',max_df=0.85)),
            ('tfidf', TfidfTransformer()),
            ('mnb', MultinomialNB(alpha= 0.05,fit_prior= False))])
            #('clf', RandomForestClassifier())])

In [21]:
# perform model training
rf.fit(X_train_final['processed_text'].values, y_train_final.values)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/utils/validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.85,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents='ascii',
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('mnb',
                 MultinomialNB(alpha=0.05, class_prior=None, fit_prior=False))],
         verbose=False)

In [22]:
# model prediction
result=X_test_final
y_pred = rf.predict(X_test_final['processed_text'].values)

result['original_cat']= y_test_final.values
result['predicted_cat'] = y_pred
result['prediction_cat_confscore'] = np.round_(np.max(rf.predict_proba(X_test_final['processed_text']), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(y_pred,y_test_final),'precision_score':precision_score(y_pred,y_test_final,average='weighted'),'recall_score':recall_score(y_pred,y_test_final,average='weighted')
,'f1_score':f1_score(y_pred,y_test_final,average='weighted')}

result['confusion_matrix'] = str(output)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', aver

In [23]:
result['confusion_matrix'] [5:6].values

array(["{'accuracy': 0.7195767195767195, 'precision_score': 0.7483143392711649, 'recall_score': 0.7195767195767195, 'f1_score': 0.7250990520577054}"],
      dtype=object)

In [24]:
#array(["{'accuracy': 0.7481380408209677, 'precision_score': 0.8104281909828657, 'recall_score': 0.7481380408209677, 'f1_score': 0.772676580987015}"],
      #dtype=object)
result

,Item,Description,establishment_type,processed_text,original_cat,predicted_cat,prediction_cat_confscore,confusion_matrix
174528,Soft Foam Earplugs,\N,pharmacy,soft foam earplug pharmacy,CAT_TPP:531,CAT_TPP:531,0.33,"{'accuracy': 0.7195767195767195, 'precision_sc..."
287305,Perrier Lime Sparkling Water 750 ml,\N,PHARMACY,perrier lime sparkling water ml pharmacy,"CAT_WATER,TRAIT_CARB,TRAIT_UNFLV_UNSWT,CONTAIN...","CAT_WATER,TRAIT_CARB,TRAIT_FLV_SWT,CONTAINER_B...",0.68,"{'accuracy': 0.7195767195767195, 'precision_sc..."
205127,Chocolate Fudge Protein Bar,\N,pharmacy,chocolate fudge protein bar pharmacy,CAT_PREPACKAGED_FOOD_SNACK_HEALTH_BARS:730,CAT_PREPACKAGED_FOOD_SNACK_HEALTH_BARS:730,0.61,"{'accuracy': 0.7195767195767195, 'precision_sc..."
106181,"SMIRNOFF, 750 mL (40.0% ABV)",CLICK TO SEE MORE FLAVORS,GROCERY,smirnoff ml abv click see flavor grocery,CAT_LIQUOR:535,CAT_BEER:533,0.40,"{'accuracy': 0.7195767195767195, 'precision_sc..."
30062,CK Mondavi Sauvignon Blanc 1.5L Bottle,\N,GROCERY,ck mondavi sauvignon blanc bottle grocery,CAT_WINE:534,CAT_WINE:534,0.98,"{'accuracy': 0.7195767195767195, 'precision_sc..."
...,...,...,...,...,...,...,...,...
315951,Glenfiddich 15 Year Old Solera Reserve Single ...,NaN,LIQUOR,glenfiddich year old solera reserve single mal...,CAT_LIQUOR:535,CAT_LIQUOR:535,1.00,"{'accuracy': 0.7195767195767195, 'precision_sc..."
64895,JOHNNIE WALKER RED LABEL SCOTCH WHISKY 375 ML,750 ml 40%\nScotland- One of the world's best ...,GROCERY,johnnie walker red label scotch whisky ml ml n...,CAT_LIQUOR:535,CAT_LIQUOR:535,1.00,"{'accuracy': 0.7195767195767195, 'precision_sc..."
20215,BUDWEISER 6PC BOTTLES 12OZ,"Known as “The King of Beers”, Budweiser was fi...",GROCERY,budweiser pc bottle oz known king beer budweis...,CAT_BEER:533,CAT_BEER:533,1.00,"{'accuracy': 0.7195767195767195, 'precision_sc..."
162822,CVS Health Vitamin A Softgels 8000IU (2400mcg)...,Vitamin A is an essential vitamin that support...,pharmacy,cv health vitamin softgels iu mcg ct vitamin e...,CAT_SUPPLEMENTS:542,CAT_SUPPLEMENTS:542,0.94,"{'accuracy': 0.7195767195767195, 'precision_sc..."


# Hyperparameter tuning

In [25]:
import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {
  'mnb__alpha': [0.01,0.05,0.1,0.2,0.3],
}
clf = GridSearchCV(rf, grid_params,n_jobs=-1,verbose=1)
clf.fit(X_train_final['processed_text'].values, y_train_final.values)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   45.5s finished
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/utils/validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Best Score:  0.7174939692303207
Best Params:  {'mnb__alpha': 0.05}


In [ ]:
import sklearn.metrics as metrics

In [ ]:
classification_report = metrics.classification_report(y_test_final, y_pred, output_dict=True)

In [ ]:
display=pd.DataFrame(classification_report).transpose()

In [ ]:
display.to_csv('classification_report.csv')

<a id='Model Saving'></a>

# Model Saving

In [43]:
import pickle
import joblib
import datetime
# save the model to disk
filename_primary= 'finalized_model.sav'
pickle.dump(rf, open(filename_primary, 'wb'))

In [19]:
import joblib
x=joblib.load('finalized_model.sav')

In [41]:
y_pred = x.predict(X_test_final['processed_text'].values)
result['original_cat']= y_test_final
result['predicted_cat'] = y_pred
result['prediction_cat_confscore'] = np.round_(np.max(rf.predict_proba(X_test_final['processed_text']), axis=1), decimals=2)
output = {'accuracy': accuracy_score(y_pred,y_test_final),'precision_score':precision_score(y_pred,y_test_final,average='weighted'),'recall_score':recall_score(y_pred,y_test_final,average='weighted')
,'f1_score':f1_score(y_pred,y_test_final,average='weighted')}
result['confusion_matrix'] = str(output)

/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jghosh2/opt/anaconda3/envs/py365/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', aver

In [42]:
result['confusion_matrix'] [5:6].values

array(["{'accuracy': 0.7481086760732513, 'precision_score': 0.7701958157552038, 'recall_score': 0.7481086760732513, 'f1_score': 0.7520608125693393}"],
      dtype=object)

<a id='Validation and Results'></a>

# Validation and Results

In [22]:
#accuracy score of the model
accuracy = x.score(X_train_final['processed_text'].values, y_train_final)
print("Accuracy = {}".format(accuracy))

Accuracy = 0.8061408567330584


# Regression test

In [23]:
#accuracy score of the model of regression
accuracy = x.score(X_train_cicd['processed_text'].values, y_train_cicd)
print("Accuracy = {}".format(accuracy))

Accuracy = 0.34068683193086197


Saving the train and test data for reference 

In [24]:
train_size = 0.8
train_end = int(len(data_df)*train_size)
df_train = data_df[:train_end]
df_test = data_df[train_end:]
train_size_cicd=0.02
train_end_cicd = int(len(data_cicd_final)*train_size_cicd)
df2_train = data_cicd_final[:train_end_cicd]
df2_test = data_cicd_final[train_end_cicd:]
df2_train = df2_train[['Item','Description','establishment_type','combined_text','processed_text','target_new']]
df2_test = df2_test[['Item','Description','establishment_type','combined_text','processed_text','target_new']]
X_train_save = df_train.append(df2_train)
X_test_save = df_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('df_traintestdata.csv')

In [43]:
# check the misclassifications
misclassifications= result.loc[result['original_cat']!=result['predicted_cat']]

In [44]:
len(misclassifications)

16781

In [45]:
misclassifications.to_csv('misclassifications.csv')

# model tagging on new dataset

In [3]:
import joblib
x=joblib.load('finalized_model.sav')

In [4]:
#read input from historical data into dataframe
df_1= pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/ml_tagged_data/before/2023-01-16 - Weekly 711 New Items.csv', encoding='utf8',engine='python')
#choose sample data from entire data
df_1 = df_1.sample(frac=1, random_state=42)
#fill blanks with ''
df_1 = df_1.fillna('')
# combine the columns Item, Description and establishment_type into one column 'combined_text'
df_1['combined_text'] = df_1[['item_name','description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df_1['processed_text'] = df_1['combined_text'].map(lambda s:preprocess_text(s)) 
print(df_1.shape)
df_1 = df_1.reset_index(drop=True)
y_pred = x.predict(df_1['processed_text'].values)
df_1['target'] = y_pred
df_1[['cat_name','cat_int']] = df_1['target'].str.split(':', expand=True)
df_1.drop('target', inplace=True, axis=1)
df_1['prediction_cat_confscore'] =np.round_(np.max(x.predict_proba(df_1['processed_text']), axis=1), decimals=2)
df_1.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/ml_tagged_data/after/2023-01-16 - Weekly 711 New Items_after_tagging.csv')

(98841, 9)


In [5]:
#read input from historical data into dataframe
df_2= pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/ml_tagged_data/before/2023-01-16 - Weekly New GroCo Menu Items.csv', encoding='utf8',engine='python')
#choose sample data from entire data
df_2 = df_2.sample(frac=1, random_state=42)
#fill blanks with ''
df_2 = df_2.fillna('')
# combine the columns Item, Description and establishment_type into one column 'combined_text'
df_2['combined_text'] = df_2[['item_name','description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df_2['processed_text'] = df_2['combined_text'].map(lambda s:preprocess_text(s)) 
print(df_2.shape)
df_2 = df_2.reset_index(drop=True)
y_pred = x.predict(df_2['processed_text'].values)
df_2['target'] = y_pred
df_2[['cat_name','cat_int']] = df_2['target'].str.split(':', expand=True)
df_2.drop('target', inplace=True, axis=1)
df_2['prediction_cat_confscore'] =np.round_(np.max(x.predict_proba(df_2['processed_text']), axis=1), decimals=2)
df_2.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/ml_tagged_data/after/2023-01-16 - Weekly New GroCo Menu Items_after_tagging.csv')
print(df_2.shape)

(132281, 9)
(132281, 12)


In [8]:
#read input from historical data into dataframe
df_3= pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/ml_tagged_data/before/2022-12-13 - INDIRECTTX-1926 - Walgreens.csv', encoding='latin-1',engine='python')
#choose sample data from entire data
df_3 = df_3.sample(frac=1, random_state=42)
#fill blanks with ''
df_3 = df_3.fillna('')
# combine the columns Item, Description and establishment_type into one column 'combined_text'
df_3['combined_text'] = df_3[['item_name','description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df_3['processed_text'] = df_3['combined_text'].map(lambda s:preprocess_text(s)) 
print(df_3.shape)
df_3 = df_3.reset_index(drop=True)
y_pred = x.predict(df_3['processed_text'].values)
df_3['target'] = y_pred
df_3[['cat_name','cat_int']] = df_3['target'].str.split(':', expand=True)
df_3.drop('target', inplace=True, axis=1)
df_3['prediction_cat_confscore'] =np.round_(np.max(x.predict_proba(df_3['processed_text']), axis=1), decimals=2)
df_3.drop('processed_text', inplace=True, axis=1)
df_3.drop('combined_text', inplace=True, axis=1)
df_3.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/ml_tagged_data/after/2022-12-13 - INDIRECTTX-1926 - Walgreens_after_tagging.csv')
print(df_3.shape)

(2317, 10)
(2317, 11)


In [3]:
#read input from historical data into dataframe
df_3= pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/bf.csv', encoding='latin-1',engine='python')
#choose sample data from entire data
df_3 = df_3.sample(frac=1, random_state=42)
#fill blanks with ''
df_3 = df_3.fillna('')
# combine the columns Item, Description and establishment_type into one column 'combined_text'
df_3['combined_text'] = df_3[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df_3['processed_text'] = df_3['combined_text'].map(lambda s:preprocess_text(s)) 


In [5]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2415 entries, 410 to 860
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Unnamed: 0                           2415 non-null   int64 
 1   CICD Run Date                        2415 non-null   object
 2   Date (BOT Sent Details to COE Team)  2415 non-null   object
 3   UniqueUUID                           2415 non-null   object
 4   store_uuid                           2415 non-null   object
 5   item_uuid                            2415 non-null   object
 6   Item                                 2415 non-null   object
 7   Description                          2415 non-null   object
 8   establishment_type                   2415 non-null   object
 9   ML CAT Name                          2415 non-null   object
 10  ML Integer                           2415 non-null   object
 11  ML Confidence Score                  2415 

In [8]:
print(df_3.shape)
df_3 = df_3.reset_index(drop=True)
y_pred = x.predict(df_3['processed_text'].values)
df_3['target'] = y_pred
df_3[['ML CAT Name','ML Integer']] = df_3['target'].str.split(':', expand=True)
df_3.drop('target', inplace=True, axis=1)
df_3['ML Confidence Score'] =np.round_(np.max(x.predict_proba(df_3['processed_text']), axis=1), decimals=2)
df_3.drop('processed_text', inplace=True, axis=1)
df_3.drop('combined_text', inplace=True, axis=1)
df_3.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/bf.csv')
print(df_3.shape)

(2415, 21)
(2415, 19)
